## INTRODUCCIÓN

En este notebook se van a explorar diversas técnicas de muestreo, focalizándose en estrategias como el muestreo estratificado, undersampling y oversampling. Estas técnicas son fundamentales en el manejo de conjuntos de datos desbalanceados para mejorar el rendimiento de los modelos de aprendizaje automático.

En primer lugar se realizan los siguientes **muestreos**:

- **Estratificado**: Divide la población en subgrupos homogéneos (estratos) y luego realiza un muestreo independiente en cada estrato.

- **Undersampling**: Reduce la cantidad de ejemplos de la clase mayoritaria en un conjunto de datos desbalanceado para equilibrar las clases.

- **Oversampling**: Se utiliza en conjuntos de datos desbalanceados para aumentar la cantidad de ejemplos de la clase minoritaria.

Posteriormente, se va a dividir el conjunto de entrenamiento en train y **validation** para tener ya los datasets separados para el posterior balanceo de los datos.

Finalmente, realizamos el **undersampling únicamente en la parte del train**, puesto que la validación no debe estar balanceada para aplicar los algoritmos y elegir los candidatos a modelos ganadores.

### Librerías

En primer lugar, se importan las librerías necesarias y se cargan los datos guardados en los notebooks previos para emplearlos en los modelos.

In [1]:
import pandas as pd 
import numpy as np
import sklearn
from sklearn.pipeline import Pipeline
from sklearn import metrics
import warnings
import category_encoders as ce
from sklearn.model_selection import StratifiedShuffleSplit
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [2]:
warnings.filterwarnings('ignore')

Cabe indicar que tras todas las transformaciones realizadas utilizaremos los siguientes 6 datasets:
- **'X_pd_fraud_red_train'**: Recoge todas las variables sin estandarizar del X train menos aquellas que no ofrecían ninguna información y las que hemos podido reducir con una combinación de ellas gracias a su alta correlación.
- **'X_pd_fraud_red_train_std'**: El dataset anterior con las variables estandarizadas para utilizarlo en modelos que requieran estandarización.
- **'y_pd_fraud_red_train'**: Recoge la variable objetivo.
- **'X_pd_fraud_sel_train'**: Dataset con 12 de las variables originales seleccionadas por los métodos de reducción de Lasso y Ridge que se utilizará para algoritmos que tienen un alto coste computacional o para facilitar el entrenamiento de otros algoritmos reduciendo la cantidad de variables a tener en cuenta.
- **'X_pd_fraud_sel_train_std'**: Dataset anterior estandarizado.
- **'y_pd_fraud_sel_train'**: Recoge la variable objetivo.

Los datasets de test no se van a leer en este notebook puesto que no son susceptibles de balanceo ni tampoco se utilizan para la separación del train en train y validation.

A continuación se leen los csv que recogen los datasets de interés:

In [3]:
X_pd_fraud_red_train = pd.read_csv("data/X_pd_fraud_red_train.csv")
X_pd_fraud_red_train_std = pd.read_csv("data/X_pd_fraud_red_train_std.csv")
y_pd_fraud_red_train = pd.read_csv("data/y_pd_fraud_red_train.csv")
X_pd_fraud_sel_train = pd.read_csv("data/X_pd_fraud_sel_train.csv")
X_pd_fraud_sel_train_std = pd.read_csv("data/X_pd_fraud_sel_train_std.csv")
y_pd_fraud_sel_train = pd.read_csv("data/y_pd_fraud_sel_train.csv")

Eliminamos la variable "Unnamed: 0" de todos los conjuntos de datos puesto que se ha creado automáticamente al guardarlos en el notebook anterior.

In [4]:
X_pd_fraud_red_train = X_pd_fraud_red_train.drop(columns=["Unnamed: 0"])
X_pd_fraud_red_train_std = X_pd_fraud_red_train_std.drop(columns=["Unnamed: 0"])
y_pd_fraud_red_train = y_pd_fraud_red_train.drop(columns=["Unnamed: 0"])
X_pd_fraud_sel_train = X_pd_fraud_sel_train.drop(columns=["Unnamed: 0"])
X_pd_fraud_sel_train_std = X_pd_fraud_sel_train_std.drop(columns=["Unnamed: 0"])
y_pd_fraud_sel_train = y_pd_fraud_sel_train.drop(columns=["Unnamed: 0"])

# Muestreos
Todos los muestreos a realizar a continuación se van a ejecutar para los datasets de entrenamiento cargados con anterioridad de X e y: 
  - 'X_pd_fraud_red_train'
  - 'y_pd_fraud_red_train'
  - 'X_pd_fraud_red_train_std'
  - 'X_pd_fraud_sel_train'
  - 'y_pd_fraud_sel_train'
  - 'X_pd_fraud_sel_train_std'

## Muestreo estratificado
[Información adicional](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html)

El muestreo estratificado es una técnica en aprendizaje automático que implica dividir un conjunto de datos en subgrupos, llamados estratos, y luego realizar un muestreo independiente en cada estrato. El objetivo principal es garantizar que cada subgrupo esté representado de manera proporcional en la muestra final, lo que ayuda a evitar sesgos y a capturar la variabilidad presente en cada estrato. 

![texto](https://enciclopediaeconomica.com/wp-content/uploads/2019/03/Muestreo-estratificado.jpg)

A continuación se define una función para realizar el estratificado, la cual toma un conjunto de características (X) y una variable objetivo (y), y realiza una división estratificada en conjuntos de entrenamiento y validación. Posteriormente, muestra las proporciones de clases en el conjunto original y en el conjunto de entrenamiento estratificado. Finalmente, devuelve los conjuntos de características y variables objetivo para el conjunto de entrenamiento y validación.

- n_splits=1, puesto que se hace una única división
- test_size=0.2, puesto que se está asignando el 20% de los datos al conjunto de prueba.
- random_state=42, elección arbitraria de la semilla.

In [5]:
def stratified_splitter(X, y):
    stratified_splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

    for train_index, val_index in stratified_splitter.split(X, y):
        X_train_stratified, X_val_stratified = X.iloc[train_index], X.iloc[val_index]
        y_train_stratified, y_val_stratified = y.iloc[train_index], y.iloc[val_index]

    print("Proporciones de clases en el conjunto original:")
    print(y.value_counts(normalize=True))

    print("\nProporciones de clases en el conjunto de entrenamiento estratificado:")
    print(y_train_stratified.value_counts(normalize=True))

    return X_train_stratified, X_val_stratified, y_train_stratified, y_val_stratified

A continuación se realizará una división estratificada del conjunto de datos 'X_train_red' e 'y_train_red' en conjuntos de entrenamiento y validación. Además, imprimirá las proporciones de clases en el conjunto original y en el conjunto de entrenamiento estratificado.

In [6]:
stratified_splitter(X_pd_fraud_red_train, y_pd_fraud_red_train) 

Proporciones de clases en el conjunto original:
fraud_bool
0             0.988971
1             0.011029
Name: proportion, dtype: float64

Proporciones de clases en el conjunto de entrenamiento estratificado:
fraud_bool
0             0.988972
1             0.011028
Name: proportion, dtype: float64


(        income  name_email_similarity  prev_address_months_count  \
 117311     0.2               0.550879                       -1.0   
 37518      0.8               0.672267                       -1.0   
 494826     0.1               0.933442                       24.0   
 220779     0.9               0.841815                       -1.0   
 296148     0.9               0.196882                       -1.0   
 ...        ...                    ...                        ...   
 758688     0.9               0.500027                       26.0   
 165490     0.8               0.108229                       -1.0   
 645057     0.6               0.500067                       50.0   
 716104     0.9               0.081564                       -1.0   
 425514     0.9               0.523258                       -1.0   
 
         current_address_months_count  customer_age  days_since_request  \
 117311                          55.0            30            0.009349   
 37518              

A continuación, se realizará el mismo procedimiento con 'X_train_red_std' e 'y_train_red'; es decir, con el conjunto de características X reduction estandarizadas:

In [7]:
stratified_splitter(X_pd_fraud_red_train_std, y_pd_fraud_red_train)

Proporciones de clases en el conjunto original:
fraud_bool
0             0.988971
1             0.011029
Name: proportion, dtype: float64

Proporciones de clases en el conjunto de entrenamiento estratificado:
fraud_bool
0             0.988972
1             0.011028
Name: proportion, dtype: float64


(          income  name_email_similarity  prev_address_months_count  \
 117311 -1.248818               0.198493                  -0.402198   
 37518   0.817528               0.618318                  -0.402198   
 494826 -1.593209               1.521598                   0.165604   
 220779  1.161919               1.204704                  -0.402198   
 296148  1.161919              -1.025819                  -0.402198   
 ...          ...                    ...                        ...   
 758688  1.161919               0.022619                   0.211028   
 165490  0.817528              -1.332429                  -0.402198   
 645057  0.128746               0.022758                   0.756117   
 716104  1.161919              -1.424649                  -0.402198   
 425514  1.161919               0.102965                  -0.402198   
 
         current_address_months_count  customer_age  days_since_request  \
 117311                     -0.357989     -0.307116           -0.188598

A continuación, se realizará el mismo procedimiento con 'X_train_sel' e 'y_train_sel':

In [8]:
stratified_splitter(X_pd_fraud_sel_train, y_pd_fraud_sel_train)

Proporciones de clases en el conjunto original:
fraud_bool
0             0.988971
1             0.011029
Name: proportion, dtype: float64

Proporciones de clases en el conjunto de entrenamiento estratificado:
fraud_bool
0             0.988972
1             0.011028
Name: proportion, dtype: float64


(        income  name_email_similarity  prev_address_months_count  \
 117313     0.8               0.053698                       27.0   
 37527      0.3               0.998677                       11.0   
 494750     0.9               0.737958                       -1.0   
 220781     0.3               0.403609                      236.0   
 296125     0.2               0.169192                      108.0   
 ...        ...                    ...                        ...   
 758716     0.9               0.491015                       -1.0   
 165507     0.8               0.380571                      103.0   
 645005     0.9               0.066943                       -1.0   
 716094     0.7               0.733941                       -1.0   
 425458     0.1               0.038410                       48.0   
 
         customer_age  employment_status  credit_risk_score  email_is_free  \
 117313            30                  0               90.0              0   
 37527        

Por último, se realizará el mismo procedimiento con 'X_train_sel_std' e 'y_train_sel'; es decir, con el conjunto de características X selection estandarizadas:

In [9]:
stratified_splitter(X_pd_fraud_sel_train_std, y_pd_fraud_sel_train)

Proporciones de clases en el conjunto original:
fraud_bool
0             0.988971
1             0.011029
Name: proportion, dtype: float64

Proporciones de clases en el conjunto de entrenamiento estratificado:
fraud_bool
0             0.988972
1             0.011028
Name: proportion, dtype: float64


(          income  name_email_similarity  prev_address_months_count  \
 117313  0.817723              -1.522365                   0.233321   
 37527  -0.903724               1.747690                  -0.129826   
 494750  1.162013               0.845482                  -0.402185   
 220781 -0.903724              -0.311513                   4.976919   
 296125 -1.248014              -1.122706                   2.071749   
 ...          ...                    ...                        ...   
 758716  1.162013              -0.009052                  -0.402185   
 165507  0.817723              -0.391238                   1.958266   
 645005  1.162013              -1.476532                  -0.402185   
 716094  0.473434               0.831582                  -0.402185   
 425458 -1.592303              -1.575267                   0.709950   
 
         customer_age  employment_status  credit_risk_score  email_is_free  \
 117313     -0.306876          -0.474390          -0.588238      -1.

## Métodos Undersampling y Oversampling

![texto](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*7xf9e1EaoK5n05izIFBouA.png)

## Undersampling
[Información adicional](https://imbalanced-learn.org/stable/references/generated/imblearn.under_sampling.RandomUnderSampler.html)

El undersampling es una técnica de preprocesamiento de datos que aborda el desequilibrio de clases en conjuntos de datos, especialmente cuando hay una clase mayoritaria y una o más clases minoritarias. Consiste en reducir la proporción de la clase mayoritaria eliminando aleatoria o selectivamente ejemplos de esa clase. Esto ayuda a equilibrar la distribución de clases y evita que los modelos de aprendizaje automático se sesguen hacia la clase mayoritaria, mejorando así la capacidad del modelo para predecir la clase minoritaria. 

La función definida en la siguiente línea de código 'undersample_data(X, y)' realiza un submuestreo (undersampling) de los datos utilizando la técnica Random UnderSampler. El submuestreo aleatorio se realiza para igualar el número de muestras de cada clase en la variable objetivo (y):

In [10]:
def undersample_data(X, y):
    rus = RandomUnderSampler(random_state=42)
    X_resampled, y_resampled = rus.fit_resample(X, y)

    print("Proporciones de clases en y_resampled después de undersampling:")
    print(y_resampled.value_counts())

    return X_resampled, y_resampled

A continuación, se aplica esta función a 'X_train_red' e 'y_train_red' para que contengan los conjuntos de datos submuestreados, donde se ha reducido el número de muestras de la clase mayoritaria para igualar el número de muestras de la clase minoritaria. 

In [11]:
X_train_red_und_total, y_train_red_und_total = undersample_data(X_pd_fraud_red_train, y_pd_fraud_red_train)

Proporciones de clases en y_resampled después de undersampling:
fraud_bool
0             8823
1             8823
Name: count, dtype: int64


A continuación, se realiza el mismo procedimiento con 'X_train_red_std' e 'y_train_red':

In [12]:
X_train_red_std_und_total, y_train_red_und_total = undersample_data(X_pd_fraud_red_train_std, y_pd_fraud_red_train)

Proporciones de clases en y_resampled después de undersampling:
fraud_bool
0             8823
1             8823
Name: count, dtype: int64


A continuación, se realiza el mismo procedimiento con 'X_train_sel' e 'y_train_sel':

In [13]:
X_train_sel_und_total, y_train_sel_und_total = undersample_data(X_pd_fraud_sel_train, y_pd_fraud_sel_train)

Proporciones de clases en y_resampled después de undersampling:
fraud_bool
0             8823
1             8823
Name: count, dtype: int64


Por último, se realiza el mismo procedimiento con 'X_train_sel_std' e 'y_train_sel':

In [14]:
X_train_sel_std_und_total, y_train_sel_und_total = undersample_data(X_pd_fraud_sel_train_std, y_pd_fraud_sel_train)

Proporciones de clases en y_resampled después de undersampling:
fraud_bool
0             8823
1             8823
Name: count, dtype: int64


## Oversampling
[Información adicional](https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTE.html)

El oversampling es una técnica de preprocesamiento de datos utilizada para abordar el desequilibrio de clases en conjuntos de datos. Se enfoca en aumentar la proporción de la clase minoritaria generando ejemplos adicionales de esa clase. Esto ayuda a equilibrar la distribución de clases, especialmente cuando hay menos ejemplos de la clase minoritaria. El objetivo es mejorar la capacidad del modelo de aprendizaje automático para predecir la clase minoritaria, aunque se debe tener cuidado para evitar el sobreajuste y la introducción de ruido en el conjunto de datos.

A continuación se define la función 'oversample_data' que ealiza un sobre-muestreo (oversampling) de los datos utilizando la técnica SMOTE (Synthetic Minority Over-sampling Technique). El sobre-muestreo se realiza generando muestras sintéticas para la clase minoritaria, lo que ayuda a equilibrar la distribución de clases en la variable objetivo (y):

In [15]:
def oversample_data(X, y):
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X, y)

    print("Clases en y_resampled después de oversampling:")
    print(y_resampled.value_counts())

    return X_resampled, y_resampled

A continuación, se realiza el oversampling sobre los conjuntos de datos de 'X_train_red' e 'y_train_red' proporcionando conjuntos que estarán equilibrados en términos de clases después de aplicar la técnica de SMOTE para generar muestras sintéticas de la clase minoritaria:

In [16]:
oversample_data(X_pd_fraud_red_train, y_pd_fraud_red_train)

Clases en y_resampled después de oversampling:
fraud_bool
0             791177
1             791177
Name: count, dtype: int64


(           income  name_email_similarity  prev_address_months_count  \
 0        0.100000               0.137525                       -1.0   
 1        0.800000               0.544001                       -1.0   
 2        0.900000               0.691325                      113.0   
 3        0.600000               0.476041                       -1.0   
 4        0.800000               0.896029                      216.0   
 ...           ...                    ...                        ...   
 1582349  0.642443               0.425144                       -1.0   
 1582350  0.717807               0.404412                       -1.0   
 1582351  0.900000               0.565406                       -1.0   
 1582352  0.154197               0.165366                       -1.0   
 1582353  0.874377               0.650374                       -1.0   
 
          current_address_months_count  customer_age  days_since_request  \
 0                          159.000000            50      

A continuación, se realiza el mismo procedimiento con 'X_train_red_std' e 'y_train_red':

In [17]:
oversample_data(X_pd_fraud_red_train_std, y_pd_fraud_red_train)

Clases en y_resampled después de oversampling:
fraud_bool
0             791177
1             791177
Name: count, dtype: int64


(           income  name_email_similarity  prev_address_months_count  \
 0       -1.593209              -1.231107                  -0.402198   
 1        0.817528               0.174704                  -0.402198   
 2        1.161919               0.684230                   2.186977   
 3        0.128746              -0.060339                  -0.402198   
 4        0.817528               1.392206                   4.526319   
 ...           ...                    ...                        ...   
 1582349  1.088834              -1.152828                  -0.402198   
 1582350  0.128746              -1.044398                  -0.402198   
 1582351  1.041369              -0.230939                  -0.402198   
 1582352 -0.811102              -1.583130                  -0.402198   
 1582353  1.161919               1.076083                  -0.402198   
 
          current_address_months_count  customer_age  days_since_request  \
 0                            0.817848      1.356764      

A continuación, se realiza el mismo procedimiento con 'X_train_sel' e 'y_train_sel':

In [18]:
oversample_data(X_pd_fraud_sel_train, y_pd_fraud_sel_train)

Clases en y_resampled después de oversampling:
fraud_bool
0             791177
1             791177
Name: count, dtype: int64


(           income  name_email_similarity  prev_address_months_count  \
 0        0.500000               0.172538                  -1.000000   
 1        0.100000               0.632957                  -1.000000   
 2        0.100000               0.102029                  -1.000000   
 3        0.900000               0.270367                  -1.000000   
 4        0.800000               0.119439                  59.000000   
 ...           ...                    ...                        ...   
 1582349  0.615114               0.391997                  -1.000000   
 1582350  0.900000               0.063089                  -1.000000   
 1582351  0.835004               0.029656                 235.848912   
 1582352  0.618704               0.607810                  -1.000000   
 1582353  0.900000               0.893710                  -1.000000   
 
          customer_age  employment_status  credit_risk_score  email_is_free  \
 0                  60                  2         104.0

Por último, se realiza el mismo procedimiento con 'X_train_sel_std' e 'y_train_sel':

In [19]:
oversample_data(X_pd_fraud_sel_train_std, y_pd_fraud_sel_train)

Clases en y_resampled después de oversampling:
fraud_bool
0             791177
1             791177
Name: count, dtype: int64


(           income  name_email_similarity  prev_address_months_count  \
 0       -0.215145              -1.111126                  -0.402185   
 1       -1.592303               0.482134                  -0.402185   
 2       -1.592303              -1.355118                  -0.402185   
 3        1.162013              -0.772592                  -0.402185   
 4        0.817723              -1.294871                   0.959613   
 ...           ...                    ...                        ...   
 1582349 -1.446177               0.056881                  -0.402185   
 1582350  1.162013              -1.455872                  -0.402185   
 1582351  0.938237              -1.608559                   4.790762   
 1582352 -1.294662              -1.274177                  -0.402185   
 1582353  1.162013               1.310942                  -0.402185   
 
          customer_age  employment_status  credit_risk_score  email_is_free  \
 0            2.186684           1.086083          -0.3

[Información útil para la siguiente explicación](https://crunchingthedata.com/oversampling-vs-undersampling/)

[Más información](https://www.blog.trainindata.com/undersampling-techniques-for-imbalanced-data/)

Puesto que queremos abordar el problema de desequilibrio de clases, tras leer bastante información sobre las diferencias entre undersampling y oversampling, consideramos que es más apropiado hacer un undersampling en este dataset puesto que es un conjunto de datos muy grande.
Además, cuando una clase está sobrerrepresentada comparándola con otra, el modelo de aprendizaje automático puede tener dificultades para aprender patrones en la clase minoritaria, puesto que puede verse dominada por la clase mayoritaria (pudiendo existir sesgos hacia dicha clase). Por ello, si realizamos un undersampling y la clase mayoritaria está bien representado, no perderemos información si eliminamos algunas de las instancias.

### División del conjunto de entrenamiento en train y validation

Una vez realizados los muestreos dividimos el conjunto de train en train y validation para que nuestro conjunto train y val estén ambos balanceados. Esta división la vamos a realizar sobre el train **previamente al undersampling, es decir, sin balancear**, puesto que el validation no debe someterse a un balanceo. 
Tal y como se observa a continuación la división se realizará con la siguientes proporciones: un 20% de los datos se utilizará como conjunto de validación, y el 80% restante se utilizará como conjunto de entrenamiento.

Para los datasets de 'red' sin estandarizar:

In [20]:
X_train_red, X_red_val, y_train_red, y_red_val = train_test_split(X_pd_fraud_red_train, y_pd_fraud_red_train,
                                                   stratify= y_pd_fraud_red_train, 
                                                   test_size=0.2, random_state=12345)

Para los datasets de 'red' estandarizados:

In [21]:
X_train_red_std, X_red_val_std, y_train_red, y_red_val = train_test_split(X_pd_fraud_red_train_std, y_pd_fraud_red_train,
                                                   stratify= y_pd_fraud_red_train, 
                                                   test_size=0.2, random_state=12345)

Para los datasets de 'sel' sin estandarizar:

In [22]:
X_train_sel, X_sel_val, y_train_sel, y_sel_val = train_test_split(X_pd_fraud_sel_train, y_pd_fraud_sel_train,
                                                   stratify= y_pd_fraud_sel_train, 
                                                   test_size=0.2, random_state=12345)

Para los datasets de 'red' estandarizados:

In [23]:
X_train_sel_std, X_sel_val_std, y_train_sel, y_sel_val = train_test_split(X_pd_fraud_sel_train_std, y_pd_fraud_sel_train,
                                                   stratify= y_pd_fraud_sel_train, 
                                                   test_size=0.2, random_state=12345)

### Undersampling sobre el conjunto de train

A continuación, realizamos el **undersampling** de la parte del train únicamente, puesto que ha sido el método de balanceo elegido para este proyecto. Además, recalcar una vez más que el undersampling lo hacemos por separado ahora puesto que el validation no debe estar balanceado para realizar los modelos y elegir los mejores.

Undersampling de 'X_train_red' e 'y_train_red':

In [24]:
X_train_red_und, y_train_red_und = undersample_data(X_train_red, y_train_red)

Proporciones de clases en y_resampled después de undersampling:
fraud_bool
0             7058
1             7058
Name: count, dtype: int64


Undersampling de 'X_train_red_std' e 'y_train_red':

In [25]:
X_train_red_std_und, y_train_red_std_und = undersample_data(X_train_red_std, y_train_red)

Proporciones de clases en y_resampled después de undersampling:
fraud_bool
0             7058
1             7058
Name: count, dtype: int64


Undersampling de 'X_train_sel' e 'y_train_sel':

In [26]:
X_train_sel_und, y_train_sel_und = undersample_data(X_train_sel, y_train_sel)

Proporciones de clases en y_resampled después de undersampling:
fraud_bool
0             7058
1             7058
Name: count, dtype: int64


Undersampling de 'X_train_sel_std' e 'y_train_sel':

In [27]:
X_train_sel_std_und, y_train_sel_std_und = undersample_data(X_train_sel_std, y_train_sel)

Proporciones de clases en y_resampled después de undersampling:
fraud_bool
0             7058
1             7058
Name: count, dtype: int64


Exportamos a csv los datasets de train y val de cada undersampling realizado, así como el dataset entero de train realizado más arriba en el notebook.

In [28]:
X_train_red_und_total.to_csv("data/X_train_red_und_total.csv")
y_train_red_und_total.to_csv("data/y_train_red_und_total.csv")
X_train_red_std_und_total.to_csv("data/X_train_red_std_und_total.csv")
X_train_sel_und_total.to_csv("data/X_train_sel_und_total.csv")
y_train_sel_und_total.to_csv("data/y_train_sel_und_total.csv")
X_train_sel_std_und_total.to_csv("data/X_train_sel_std_und_total.csv")

In [29]:
X_train_red_und.to_csv("data/X_train_red_und.csv")
X_train_red_std_und.to_csv("data/X_train_red_std_und.csv")
X_red_val.to_csv("data/X_val_red.csv")
X_red_val_std.to_csv("data/X_val_red_std.csv")
y_train_red_und.to_csv("data/y_train_red_und.csv")
y_red_val.to_csv("data/y_val_red.csv")
X_train_sel_und.to_csv("data/X_train_sel_und.csv")
X_train_sel_std_und.to_csv("data/X_train_sel_std_und.csv")
X_sel_val.to_csv("data/X_val_sel.csv")
X_sel_val_std.to_csv("data/X_val_sel_std.csv")
y_train_sel_und.to_csv("data/y_train_sel_und.csv")
y_sel_val.to_csv("data/y_val_sel.csv")

### CONCLUSIONES

- En primer lugar se han llevado a cabo **los muestreos de estratificado, oversampling y undersampling** sobre los conjuntos de datos de X e y de entrenamiento:
  - 'X_pd_fraud_red_train'
  - 'y_pd_fraud_red_train'
  - 'X_pd_fraud_red_train_std'
  - 'X_pd_fraud_sel_train'
  - 'y_pd_fraud_sel_train'
  - 'X_pd_fraud_sel_train_std'

- Aunque se han realizado estas tres técnicas, **consideramos más pertinente utilizar el undersampling** puesto que es preferible para conjuntos de datos grandes, como con el que estamos trabajando. Además, en situaciones donde una clase está significativamente sobrerepresentada, el modelo de aprendizaje automático puede enfrentar desafíos al aprender patrones en la clase minoritaria, ya que podría ser ocultada por la clase mayoritaria, introduciendo posibles sesgos.

- Al optar por el undersampling, se busca **equilibrar las clases eliminando instancias de la clase mayoritaria**, asegurando así que la representación de la clase minoritaria sea más destacada durante el proceso de aprendizaje. Esta estrategia permite mejorar la capacidad del modelo para generalizar patrones en la clase minoritaria, sin sacrificar información crítica en el conjunto de datos.

- Posteriormente, se ha realizado **la separación del train en train y validation (80% y 20%, respectivamente)**.

- Finalmente, se ha realizado la técnica de balanceo del **undersampling sobre el conjunto de train únicamente** puesto que la validación no debe estar balanceada para la consecución de los pasos que se llevarán a cabo en los siguientes notebooks.